In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def mean_exp_fitness(pop_dist, fitness_list):
    return np.sum(pop_dist*np.exp(fitness_list))/np.sum(pop_dist)

def wright_fisher_probabilities(pop_dist, fitness_list):
    return np.exp(fitness_list)/mean_exp_fitness(pop_dist, fitness_list)*pop_dist/np.sum(pop_dist)

def wright_fisher_fitness_update(pop_dist, fitness_list):
    probabilities = wright_fisher_probabilities(pop_dist, fitness_list)
    shape = probabilities.shape
    probabilities.shape = shape[0]*shape[1]
    new = np.random.multinomial(np.sum(pop_dist), probabilities)
    new.shape = shape
    return new

In [3]:
fitnesses = np.transpose(np.atleast_2d(np.linspace(.05,-.05,11)))
print(fitnesses)

[[ 0.05]
 [ 0.04]
 [ 0.03]
 [ 0.02]
 [ 0.01]
 [ 0.  ]
 [-0.01]
 [-0.02]
 [-0.03]
 [-0.04]
 [-0.05]]


In [4]:
pop_dist = 10**7*np.ones((11,5)).astype('int32')
print(pop_dist)
print(np.sum(pop_dist))

[[10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]
 [10000000 10000000 10000000 10000000 10000000]]
550000000


In [5]:
probabilities=wright_fisher_probabilities(pop_dist, fitnesses)
print(probabilities)
print(np.sum(probabilities))

[[ 0.01910447  0.01910447  0.01910447  0.01910447  0.01910447]
 [ 0.01891437  0.01891437  0.01891437  0.01891437  0.01891437]
 [ 0.01872617  0.01872617  0.01872617  0.01872617  0.01872617]
 [ 0.01853984  0.01853984  0.01853984  0.01853984  0.01853984]
 [ 0.01835537  0.01835537  0.01835537  0.01835537  0.01835537]
 [ 0.01817273  0.01817273  0.01817273  0.01817273  0.01817273]
 [ 0.01799191  0.01799191  0.01799191  0.01799191  0.01799191]
 [ 0.01781289  0.01781289  0.01781289  0.01781289  0.01781289]
 [ 0.01763565  0.01763565  0.01763565  0.01763565  0.01763565]
 [ 0.01746017  0.01746017  0.01746017  0.01746017  0.01746017]
 [ 0.01728644  0.01728644  0.01728644  0.01728644  0.01728644]]
1.0


In [6]:
wag = pop_dist
for i in range(10):
    wag = wright_fisher_fitness_update(wag, fitnesses)
print(wag)

[[15679424 15684825 15696234 15693521 15678646]
 [14214940 14188126 14188820 14192847 14219198]
 [12877182 12856284 12839234 12851358 12827882]
 [11620307 11613872 11620578 11623727 11650354]
 [10499274 10520781 10506881 10507619 10524641]
 [ 9525089  9513003  9536597  9516964  9504772]
 [ 8598541  8609914  8619976  8633090  8604414]
 [ 7796165  7785066  7791260  7789730  7784199]
 [ 7041828  7042744  7057969  7039500  7044427]
 [ 6394437  6381012  6380659  6390241  6376357]
 [ 5773312  5770717  5775990  5769170  5776302]]


In [7]:
np.sum(wag)

550000000

In [8]:
def array_multinomial(N_array, Pis_array):
    if Pis_array.shape[1:] != N_array.shape:
        raise AttributeError('Pis_array must be the shape of N_array plus one additional axis in the lead')
    return_array = np.zeros_like(Pis_array, dtype='int32')
    N_remain_array = np.copy(N_array)
    prob_remain = np.ones_like(N_array, dtype='float64')
    for i in range(Pis_array.shape[0]-1):
        return_array[i,...]=np.random.binomial(N_remain_array, Pis_array[i,...]/prob_remain)
        N_remain_array -= return_array[i,...]
        prob_remain -= Pis_array[i,...]
    return_array[-1,...]=N_remain_array
    return return_array

In [14]:
N_array = np.ones((30,30),dtype='int32')*1000
print(N_array)
p_array = np.zeros((5,30,30))
p_array[0,...]=np.arange(900).reshape((30,30))*.0003
p_array[1,...]=.3
p_array[2,...]=.02
p_array[3,...]=.04
p_array[4,...]=1-p_array[0,...]-p_array[1,...]
print(p_array)

[[1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000]
 [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000]
 [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000]
 [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000]
 [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000]
 [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
  1000 1000]
 [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1

In [15]:
np.sum(p_array<0)

0

In [17]:
multidraw = array_multinomial(N_array, p_array)
print(multidraw[:,0:10,0:10])

[[[  0   0   0   2   0   1   2   3   4   5]
  [  8   7  12  10  11  11  14  15  12  15]
  [ 16  22  17   9  21  22  32  23  12  21]
  [ 31  46  33  23  23  30  26  32  23  26]
  [ 44  35  34  40  44  50  33  40  43  41]
  [ 43  45  48  49  48  52  44  63  42  67]
  [ 59  63  67  55  52  47  46  66  60  67]
  [ 65  74  73  62  64  57  67  54  61  66]
  [ 76  79  93  68  68  77  89  74  82  69]
  [ 86  65  82  84  77  78  74  81  74  83]]

 [[280 323 293 309 306 299 310 304 316 265]
  [296 290 310 281 299 288 313 271 287 304]
  [314 299 316 305 294 296 324 281 313 312]
  [293 312 299 302 297 284 295 332 293 307]
  [295 269 328 325 312 280 317 290 307 289]
  [308 297 316 286 302 341 303 313 309 289]
  [289 281 293 315 287 285 294 276 304 290]
  [291 300 308 292 286 303 282 298 321 307]
  [324 310 298 313 290 297 282 326 312 280]
  [278 298 288 302 312 310 315 294 302 318]]

 [[ 18  20  16  12  20  17  30  21  22  30]
  [ 21  26  19  26  12  23  32  19  20  13]
  [ 28  15  23  12  27  24  

In [18]:
%timeit array_multinomial(N_array, p_array)

The slowest run took 5.02 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 672 µs per loop


In [124]:
def subarray(array, axis, index):
    all = slice(None, None, None)
    max_index = len(array.shape)-1
    multislice = (all,)*axis+(index,)+(all,)*(max_index-axis)
    return array[multislice]

def stenciled_sum(big_array, stencil, axis=0):
    if stencil.shape[0] != big_array.shape[axis]:
        raise ValueError('The dimension of axis 0 in the stencil must equal the dimension of the chosen axis in big_array')
    old_shape_minus_axis = np.array(big_array.shape[0:axis] + big_array.shape[axis+1:])
    added_space = np.amax(stencil, axis=0)
    return_array = np.zeros(old_shape_minus_axis + added_space, dtype = big_array.dtype)
    for i in range(big_array.shape[axis]):
        multislice = ()
        k=0
        for j in range(0, axis):
            multislice = multislice + (slice(stencil[i,k], big_array.shape[j]+stencil[i,k], None),)
            k=k+1
        for j in range(axis+1, big_array.ndim):
            multislice = multislice + (slice(stencil[i,k], big_array.shape[j]+stencil[i,k], None),)
            k=k+1
        return_array[multislice] = return_array[multislice] + subarray(big_array, axis, i)
    return return_array

In [125]:
a = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

b = np.array([[0, 0, 2],
              [0, 2, 0],
              [2, 0, 0]])

big_array = np.array([a,b])

print(big_array.shape)

stencil = np.array([[0,0],
                    [0,1],
                    [0,2]])

stenciled_sum_array = stenciled_sum(big_array, stencil, 1)
print(stenciled_sum_array)
print(stenciled_sum_array.shape)

(2, 3, 3)
[[1 0 1 0 1]
 [0 0 6 0 0]]
(2, 5)


In [44]:
bwah = np.arange(160).reshape(2,2,10,2,2)

In [104]:
bwah.ndim

5

In [45]:
bwah

array([[[[[  0,   1],
          [  2,   3]],

         [[  4,   5],
          [  6,   7]],

         [[  8,   9],
          [ 10,  11]],

         [[ 12,  13],
          [ 14,  15]],

         [[ 16,  17],
          [ 18,  19]],

         [[ 20,  21],
          [ 22,  23]],

         [[ 24,  25],
          [ 26,  27]],

         [[ 28,  29],
          [ 30,  31]],

         [[ 32,  33],
          [ 34,  35]],

         [[ 36,  37],
          [ 38,  39]]],


        [[[ 40,  41],
          [ 42,  43]],

         [[ 44,  45],
          [ 46,  47]],

         [[ 48,  49],
          [ 50,  51]],

         [[ 52,  53],
          [ 54,  55]],

         [[ 56,  57],
          [ 58,  59]],

         [[ 60,  61],
          [ 62,  63]],

         [[ 64,  65],
          [ 66,  67]],

         [[ 68,  69],
          [ 70,  71]],

         [[ 72,  73],
          [ 74,  75]],

         [[ 76,  77],
          [ 78,  79]]]],



       [[[[ 80,  81],
          [ 82,  83]],

         [[ 84,  85],
      

In [89]:
subarray(bwah, 0, 1)

(1, slice(None, None, None), slice(None, None, None), slice(None, None, None), slice(None, None, None))


array([[[[ 80,  81],
         [ 82,  83]],

        [[ 84,  85],
         [ 86,  87]],

        [[ 88,  89],
         [ 90,  91]],

        [[ 92,  93],
         [ 94,  95]],

        [[ 96,  97],
         [ 98,  99]],

        [[100, 101],
         [102, 103]],

        [[104, 105],
         [106, 107]],

        [[108, 109],
         [110, 111]],

        [[112, 113],
         [114, 115]],

        [[116, 117],
         [118, 119]]],


       [[[120, 121],
         [122, 123]],

        [[124, 125],
         [126, 127]],

        [[128, 129],
         [130, 131]],

        [[132, 133],
         [134, 135]],

        [[136, 137],
         [138, 139]],

        [[140, 141],
         [142, 143]],

        [[144, 145],
         [146, 147]],

        [[148, 149],
         [150, 151]],

        [[152, 153],
         [154, 155]],

        [[156, 157],
         [158, 159]]]])